<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


# DAY1

### Learning Objectives
- Describe the foundational components of a neural network
- Implement a Perceptron from scratch in Python

#### Input Layer:

The input Layer is where the feature data from the dataframe are input

#### Hidden Layer:

These are the layer that exist between the input layer and output layer. You cna have one hidden layer or many hidden layers

#### Output Layer:

This is the answer/result of our neurons in our neural netoworks. These ouputs can then be used as inputs for the next layer of neurons or be the final output(s) of the neural network.

#### Neuron:

The neuron recieves inputs, multiplies the inputs by their weights, sums everyhting up, and then applies the activation function to the sum. Usually involves a continuous activation function

#### Weight:

This is the amount or positive or negative effect an input will be associated with the ending output.

#### Activation Function:

The activation function is how the neural network normalizes the results after inputs, weights, and biases have been applied within the neuron.

#### Node Map:

The node maps show how the features of the dataframe or the outputs of upper level neurons are further processed throughout the neural netowork. It shows inputs, outputs, and hidden layers visualized at a high level.

#### Perceptron:

Simply, a perceptron consists of four distinct parts. Uses a binary activation function that is either activate or not, different from a neuron

    Inputs
    Weights
    Weighted Sum
    Activation Function (Output)

Perceptrons classify data into two parts (0,1) most of the time. Perceptrons are also known as Linear Binary Classifiers


#### Inputs -> Outputs
Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?
Your Answer Here

Depending on your network, Inputs and Outputs can range arbitraily. Each input can come from an upper level neuron or the intial inputted values from a dataframe. Each input can be weighted negatively or positvely depending on whether your desired answer needs the neuron to activate negatively or positively depending how your inputted bias has shifted the activation curve up or down.


### Imports

In [1]:
!pip install category-encoders

In [47]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import category_encoders as ce

In [89]:
help(keras.wrappers.scikit_learn.KerasClassifier)

Help on class KerasClassifier in module tensorflow.python.keras.wrappers.scikit_learn:

class KerasClassifier(BaseWrapper)
 |  KerasClassifier(build_fn=None, **sk_params)
 |  
 |  Implementation of the scikit-learn classifier API for Keras.
 |  
 |  Method resolution order:
 |      KerasClassifier
 |      BaseWrapper
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  fit(self, x, y, **kwargs)
 |      Constructs a new model with `build_fn` & fit the model to `(x, y)`.
 |      
 |      Arguments:
 |          x : array-like, shape `(n_samples, n_features)`
 |              Training samples where `n_samples` is the number of samples
 |              and `n_features` is the number of features.
 |          y : array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
 |              True labels for `x`.
 |          **kwargs: dictionary arguments
 |              Legal arguments are the arguments of `Sequential.fit`
 |      
 |      Returns:
 |          history : object
 |       

In [91]:
#Load Data
df = sns.load_dataset('tips')
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [92]:
print(df.shape)
df.head()

(244, 7)


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [93]:
def prep(df, target):
    
    """
    This function will:
    1. Change "size" into a catagorical to be one hotted
    2. Add Total and Tip and put into 3 bins
    3. Split data
    4. Create X and y train/test
    5. process X train/test data by one hotting categoricals
    6. Make 'sex' a binary column
    7. return 4 df's
    """
    df['size'] = df['size'].astype(str)
    df['bill_tip_sum'] = pd.qcut(df['total_bill']+df['tip'], 3, labels=['low', 'medium', 'high'])
    df['tip_pct'] = df['tip']/df['total_bill']
    
    training, testing = train_test_split(df, test_size=.2)
    
    X_train = training.drop(columns=target)
    y_train = training[target]
    X_test = testing.drop(columns=target)
    y_test = testing[target]
    
    processor = make_pipeline(
        ce.OneHotEncoder(use_cat_names=True),  
        SimpleImputer(strategy='median'), # Use when a normalized dataframe is needed
        StandardScaler() # Use when a normalized dataframe is needed
    )
    
    gender = {'Female': 0, 'Male': 1}
    y_train = y_train.map(gender)
    y_test = y_test.map(gender)
    
    X_process_train = processor.fit_transform(X_train)
    X_process_test = processor.transform(X_test)
    
    return X_process_train,y_train, X_process_test, y_test

In [94]:
X_train, y_train, X_test, y_test = prep(df, 'sex')
print(X_train.shape) 
print(X_test.shape) 
print(y_train.shape) 
print(y_test.shape)
# X_train.head()

(195, 20)
(49, 20)
(195,)
(49,)


In [11]:
class NNet:
    def __init__(self):
        
        # Inputs must be == to number of features
        self.inputs = 20
        # Only one output node b/c only trying to predict one thing
        self.outputNodes = 1
        
        self.weights = np.random.rand(self.inputs, self.outputNodes)
     
    # Squishify
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    # Create 0 or 1 from prediced activated output
    def binary(self, X):
        binary = self.feed_forward(X)
        binary = [1 if x > .9999 else 0 for x in binary]
        return binary
    
     
    def feed_forward(self, X):
        """Calculate the NNet inference using the feed forward, aka predict """
        
        # Combining  inputs and weights in a weighted sum
        self.input_sum = np.dot(X, self.weights)
        
        # Apply activation function to the weighted sum
        self.output_activated = self.sigmoid(self.input_sum)
        
        return self.output_activated

In [12]:
nn = NNet()

In [13]:
y_pred1 = nn.binary(X_train)
score = accuracy_score(y_train, y_pred1)

y_pred2 = nn.binary(X_test)
score2 = accuracy_score(y_test, y_pred2)

print(f"Mean baseline for our target(Males) is {round(df['sex'].value_counts(normalize=True)[0]*100, 2)}%")
print(f"The accuracy of the train is {round(score*100, 2)}%")
print(f"The accuracy of the test is {round(score2*100, 2)}%")

Mean baseline for our target(Males) is 64.34%
The accuracy of the train is 66.15%
The accuracy of the test is 61.22%


# Day 2

### Learning Objectives
- Explain the intuition behind backproprogation
- Implement gradient descent + backproprogation on a feedforward neural network

In [14]:
# I want activations that correspond to negative weights to be lower
# and activations that correspond to positive weights to be higher

class NNetbackprop:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 20
        self.hiddenNodes = 3
        self.outputNodes = 1

        # Initial Weights
        # 20x3 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
 
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
        
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        # Calculate the error, the diffrence between true y value and the predicted
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        # Which direction do we want to go 
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error
        # Applying the  o-delta/correction to weights2 transformed
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # How much of that "far off" can explained by the input => hidden
        # Apply sigmoid derivative to the error
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        # Applying adjustments to the weights
        self.weights1 += X.T.dot(self.z2_delta)
        
        # Adjustment to second set of weights (hidden => output)
        # Applying adjustments to the weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)

### Preprocessing

In [15]:
def y_input(x):
    "Changes our y into a list of individual arrays"
    y_list = []
    for x in y_train:
        new = np.array([x])
        y_list.append(new)
    return y_list

In [16]:
nnbp = NNetbackprop()

### Backproprogation

In [18]:
# ---1st ERROR---
# Apply sigmoid derivative to the error
# Which direction do we want to go
# self.o_delta = self.o_error * sigmoidprime(o)
#How much more sigmoid activation would have pushed us towards the right answer

nnbp.o_error

array([[ 0.23762945],
       [ 0.23649258],
       [ 0.23783894],
       [ 0.23884257],
       [ 0.23626209],
       [ 0.23639935],
       [-0.76188718],
       [-0.76364747],
       [ 0.23781503],
       [ 0.23741653],
       [ 0.2362694 ],
       [ 0.23708812],
       [-0.76346891],
       [-0.76229826],
       [ 0.23741541],
       [ 0.23647091],
       [ 0.23874589],
       [ 0.23708745],
       [-0.7625914 ],
       [ 0.23845791],
       [-0.76302876],
       [ 0.23627397],
       [-0.76232617],
       [ 0.23652943],
       [ 0.23648952],
       [ 0.23675915],
       [ 0.23673362],
       [ 0.23781947],
       [-0.7604533 ],
       [ 0.23649259],
       [ 0.23721636],
       [-0.76379987],
       [-0.76376102],
       [-0.76254848],
       [ 0.23629376],
       [-0.76375964],
       [-0.76198272],
       [ 0.23730545],
       [-0.7621048 ],
       [-0.76251685],
       [ 0.23778928],
       [-0.76262617],
       [ 0.23731801],
       [ 0.23766567],
       [-0.76259665],
       [ 0

In [19]:
# Apply sigmoid derivative to the error
# Which direction do we want to go
# self.o_delta = self.o_error * sigmoidprime(o)

nnbp.o_delta.shape

(195, 1)

In [20]:
# z2 error
# Applying the  o-delta/correction to weights2 transformed
# These are the errors from the output to the hidden layer

nnbp.z2_error.shape

(195, 3)

In [21]:
# How much on the sigmoid curve we want to move
# Being the delta this is the direction we will be traveling
# For each observation, how much more sigmoid activation from this layer would have 
# pushed us towards the right answer?

nnbp.z2_delta.shape

(195, 3)

In [22]:
#Calculation to update the weights
X_train.T.dot(nnbp.z2_delta)

,0,1,2
total_bill,-0.108696,-2.073757e-04,-0.212623
tip,-0.019364,-3.530924e-05,-0.039501
smoker_Yes,-0.000916,-3.081483e-05,-0.004601
smoker_No,-0.007802,-3.900516e-06,-0.012292
day_Thur,-0.006838,-1.965600e-06,-0.007879
day_Fri,-0.001292,-3.153265e-05,-0.004468
day_Sat,-0.000830,-1.368510e-06,-0.004521
day_Sun,0.000243,1.514064e-07,-0.000026
time_Lunch,-0.007400,-1.956066e-06,-0.008618
time_Dinner,-0.001318,-3.275928e-05,-0.008275


In [23]:
# Update hidden layer weights

nnbp.activated_hidden.T.dot(nnbp.o_delta)

array([[-3.50861178],
       [-3.75954309],
       [-3.72649562]])

In [24]:
# Train my 'net
nnbp = NNetbackprop()

# Number of Epochs / Iterations
for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X_train)
        print('Actual Output: \n', ytrain)
        print('Predicted Output: \n', str(nn.feed_forward(X_train)))
        print("Loss: \n", str(np.mean(np.square(ytrain - nnbp.feed_forward(X_train)))))
    nnbp.train(X_train,ytrain)

+---------EPOCH 1---------+
Input: 
      total_bill   tip  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
50        12.54  2.50           0          1         0        0        0   
15        21.58  3.92           0          1         0        0        0   
217       11.59  1.50           1          0         0        0        1   
235       10.07  1.25           0          1         0        0        1   
88        24.71  5.85           0          1         1        0        0   
46        22.23  5.00           0          1         0        0        0   
135        8.51  1.25           0          1         1        0        0   
114       25.71  4.00           0          1         0        0        0   
236       12.60  1.00           1          0         0        0        1   
110       14.00  3.00           0          1         0        0        1   
184       40.55  3.00           1          0         0        0        0   
76        17.92  3.08           1          0       

# Day 3

### Learning Objectives
- Introduce the Keras Sequential Model API
- Learn How to Select Model Architecture
- Discuss the trade-off between various activation functions

In [27]:
#Model
model = Sequential()

#Input
model.add(Dense(16, input_dim=20, activation='relu'))

#Hidden
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, kernel_initializer='normal', activation='relu'))

#Output
model.add(Dense(1, activation='linear'))

#Compile
model.compile(loss='mean_squared_error',
              metrics=['mean_squared_error'],
              optimizer='adam')

#Fit & Evaluate
history = model.fit(X_train, y_train, epochs=100, verbose=False, validation_split=.1)
scores = model.evaluate(X_test, y_test, verbose=0)

In [28]:
print(f"The MSE of our neural net is ${scores[1]}")
print(f"The RMSE of our neural net is ${round(np.sqrt(scores[1]), 2)}")

The MSE of our neural net is $0.26597440242767334
The RMSE of our neural net is $0.5199999809265137


### Activation Functions

#### Step Function

- Binary activation function. 
- Updating weights through back proprogation is impossible due to all or nothing nature
- Since the derivative has no slope it cannot update weights


#### Linear Function

- Passes the signal onto the next layer by a constant factor
- Derivative is of a linear activation is horizontal which would mean we should update all weights by a constant
amount everytime
- Only used for very simple tasks where intepretability is important

#### Sigmoid Function

- Sigmoid and its derivatives are usually better at classification problems
- Great activation function since its continuously differentiable
- Slope reaches 0 quickly when moving away from zero
- Higher slope around 0 pushes our y more quickly to one of the extremes
- Useful for binary classification

#### Tanh Function

- Doesnt get so flat when moving away from zero
- A little steeper in the middle than sigmoid
- Created by scaling y by 2 in the y dimmension and subtracting 1 from all values
- Same drawbacks as sigmoid like diminishing flat gradients when moving from zero
- Derivative higher at 0 causing weights to move to extremes faster

#### ReLu Function (Rectified Linear Units)

- Typicaly see ReLu for initial layers
- Generally better at obtaining optimal model fit
- Commonly used as activatoin functions in neural networks
- Doesnt activate when neuron outputs a negative signal but passes on positive signal
- Derivative looks like a step function
- Turns off a portion of our less important neurons which decreases computational load
- Can lead to dead neurons since negatively weighted neurons wont activate
- Might want to to upadte negatively initialized weights

#### Leaky ReLu

- Avoids having a gradient of 0 on the left side of the derivative funcion
- Even 'dead' neurons have a chance of being revived w/ enough iterations
- Leaky side slope can even be tuned as a hyper parameter

#### Softmax Funtion

- Good for multiclassification problems
- Takes any set of inputs and translates them into probabilities that sum to 1
- Can ouput a list of outputs and translate them into probabilities that sum to 1

# DAY 4

### Learning Objectives
- Describe the major hyperparameters to tune
- Implement an exeriment tracking framework
- Search the hyperameter space using RandomSearch (Optional)

In [9]:
# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 75
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size,
          
         )

Train on 195 samples, validate on 49 samples
Epoch 1/75
195/195 [==============================] - 0s 724us/sample - loss: 0.3048 - mse: 0.3048 - mae: 0.4815 - val_loss: 0.2905 - val_mse: 0.2905 - val_mae: 0.5152
Epoch 2/75
195/195 [==============================] - 0s 140us/sample - loss: 0.2373 - mse: 0.2373 - mae: 0.4455 - val_loss: 0.3449 - val_mse: 0.3449 - val_mae: 0.4997
Epoch 3/75
195/195 [==============================] - 0s 148us/sample - loss: 0.2595 - mse: 0.2595 - mae: 0.4523 - val_loss: 0.2598 - val_mse: 0.2598 - val_mae: 0.4973
Epoch 4/75
195/195 [==============================] - 0s 144us/sample - loss: 0.2183 - mse: 0.2183 - mae: 0.4224 - val_loss: 0.2836 - val_mse: 0.2836 - val_mae: 0.4908
Epoch 5/75
195/195 [==============================] - 0s 177us/sample - loss: 0.2102 - mse: 0.2102 - mae: 0.4093 - val_loss: 0.3378 - val_mse: 0.3378 - val_mae: 0.4863
Epoch 6/75
195/195 [==============================] - 0s 153us/sample - loss: 0.2905 - mse: 0.2905 - mae: 0.4552 - 

In [76]:
def prepro(df, target):
    
    df['size'] = df['size'].astype(str)
    df['bill_tip_sum'] = pd.qcut(df['total_bill']+df['tip'], 3, labels=['low', 'medium', 'high'])
    df['tip_pct'] = df['tip']/df['total_bill']
    
    X = df.drop(columns=target)
    y = df.pop(target)
    
    processor = make_pipeline(
        ce.OneHotEncoder(use_cat_names=True),  
#         SimpleImputer(strategy='median'), # Use when a normalized dataframe is needed
#         StandardScaler() # Use when a normalized dataframe is needed
    )
    
    gender = {'Female': 0, 'Male': 1}
    y = y.map(gender)
    
    X_process_train = processor.fit_transform(X)
#     y_process_train = processor.fit(y)
    
#     return X_process_train, y_process_train
    return X_process_train, y

In [77]:
df = sns.load_dataset('tips')

In [78]:
X, y = prepro(df, 'sex')

In [82]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim =20, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, workers=3)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=3)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/Users/mph/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.6434426278364463 using {'batch_size': 40, 'epochs': 50}
Means: 0.6393442710892099, Stdev: 0.09631531129821289 with: {'batch_size': 10, 'epochs': 50}
Means: 0.6311475546633611, Stdev: 0.09360312413242985 with: {'batch_size': 20, 'epochs': 50}
Means: 0.6434426278364463, Stdev: 0.10033392319406603 with: {'batch_size': 40, 'epochs': 50}
Means: 0.6270491751979609, Stdev: 0.09716729483539199 with: {'batch_size': 60, 'epochs': 50}
Means: 0.6270491808164315, Stdev: 0.08546382989245388 with: {'batch_size': 80, 'epochs': 50}
Means: 0.6434426278364463, Stdev: 0.10033392319406603 with: {'batch_size': 100, 'epochs': 50}


In [83]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

# define the grid search parameters
param_grid = {'epochs': [10, 20, 40, 80],
              'batch_size': [32, 64, 128, 256],
              'validation_split': [.1, .2, .3, .4],
              'shuffle': [True, False],             
}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=5)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/Users/mph/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.6885245977366556 using {'batch_size': 32, 'epochs': 10, 'shuffle': False, 'validation_split': 0.4}
Means: 0.5983606644096922, Stdev: 0.1717913509655654 with: {'batch_size': 32, 'epochs': 10, 'shuffle': True, 'validation_split': 0.1}
Means: 0.6434426280807276, Stdev: 0.13945502288792438 with: {'batch_size': 32, 'epochs': 10, 'shuffle': True, 'validation_split': 0.2}
Means: 0.5655737671940053, Stdev: 0.1532002233259814 with: {'batch_size': 32, 'epochs': 10, 'shuffle': True, 'validation_split': 0.3}
Means: 0.5040983628542697, Stdev: 0.14868668325568637 with: {'batch_size': 32, 'epochs': 10, 'shuffle': True, 'validation_split': 0.4}
Means: 0.6106557345292607, Stdev: 0.12205832397954682 with: {'batch_size': 32, 'epochs': 10, 'shuffle': False, 'validation_split': 0.1}
Means: 0.6270491813049942, Stdev: 0.1298315601230707 with: {'batch_size': 32, 'epochs': 10, 'shuffle': False, 'validation_split': 0.2}
Means: 0.6188524582835494, Stdev: 0.1538867219463314 with: {'batch_size': 32, 'epoch

In [87]:
print(grid_result.best_params_)
print(grid_result.best_score_)

{'batch_size': 32, 'epochs': 10, 'shuffle': False, 'validation_split': 0.4}
0.6885245977366556


## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?

# Assignment

In [14]:
df2 = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [24]:
print(df2.shape)
df2.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [21]:
df2.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [25]:
df2.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [26]:
df2.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,8111-RKSPX,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


(7032, 21)

In [46]:
def prep2(df, target):
    
    
    # Change Total Charges to float and drop NaN
    rows = df[df['TotalCharges'] == " "]
    df = df.drop(rows.index, axis=0)
    df['TotalCharges'] = df['TotalCharges'].astype(float)
    
    # Split
    training, testing = train_test_split(df, test_size=.2)
    
    # Make X, y and drop appropriate columns and high catagoricals
    X_train = training.drop(columns=[target, 'customerID'])
    y_train = training[target]
    X_test = testing.drop(columns=[target, 'customerID'])
    y_test = testing[target]
    
    
    processor = make_pipeline(
        ce.OneHotEncoder(use_cat_names=True),  
#         SimpleImputer(strategy='median'), # Use when a normalized dataframe is needed
#         StandardScaler() # Use when a normalized dataframe is needed
    )
    
    binary = {'No': 0, 'Yes': 1}
    y_train = y_train.map(binary)
    y_test = y_test.map(binary)
    
    X_process_train = processor.fit_transform(X_train)
    X_process_test = processor.transform(X_test)
    
    return X_process_train,y_train, X_process_test, y_test

In [47]:
Xtrain, ytrain, Xtest, ytest = prep2(df2, 'Churn')

In [48]:
print(Xtrain.shape)
Xtrain.head()

(5625, 45)


,gender_Female,gender_Male,SeniorCitizen,Partner_Yes,Partner_No,Dependents_No,Dependents_Yes,tenure,PhoneService_Yes,PhoneService_No,...,Contract_Two year,Contract_One year,PaperlessBilling_Yes,PaperlessBilling_No,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Mailed check,MonthlyCharges,TotalCharges
53,1,0,1,1,0,1,0,8,1,0,...,0,0,1,0,1,0,0,0,80.65,633.30
3785,0,1,1,1,0,1,0,10,1,0,...,0,0,1,0,0,1,0,0,89.80,914.30
5366,1,0,0,1,0,0,1,66,1,0,...,1,0,1,0,0,1,0,0,59.75,3996.80
3426,0,1,0,0,1,1,0,1,1,0,...,0,0,1,0,0,1,0,0,69.80,69.80
3104,0,1,1,0,1,1,0,9,1,0,...,0,0,1,0,0,1,0,0,74.05,678.45


In [57]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=45, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20, 40, 60, 100]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4)
grid_result = grid.fit(Xtrain, ytrain)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.8012444376945496 using {'batch_size': 100, 'epochs': 60}
Means: 0.7957333326339722, Stdev: 0.010675538021228769 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7278222242991129, Stdev: 0.05588617371486254 with: {'batch_size': 10, 'epochs': 40}
Means: 0.7797333399454752, Stdev: 0.012866481979977896 with: {'batch_size': 10, 'epochs': 60}
Means: 0.7914666732152303, Stdev: 0.005134067389353745 with: {'batch_size': 10, 'epochs': 100}
Means: 0.7921777764956156, Stdev: 0.008821095438618167 with: {'batch_size': 20, 'epochs': 20}
Means: 0.781511127948761, Stdev: 0.021361459277727535 with: {'batch_size': 20, 'epochs': 40}
Means: 0.793066680431366, Stdev: 0.012138453802542713 with: {'batch_size': 20, 'epochs': 60}
Means: 0.7847111225128174, Stdev: 0.019186834908845082 with: {'batch_size': 20, 'epochs': 100}
Means: 0.7989333271980286, Stdev: 0.009420528921098374 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7756444414456686, Stdev: 0.01702960965129327 with: {'batch_size': 40, 'epochs': 4

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?